In [1]:
# app.py
# -------------------------------------------------------------------
# Dash app: 10 Supermarket Products (1 = Our Brand + 9 Competitors)
#   - KPI panel for our product
#   - Sales forecast chart (user picks any product from a dropdown)
#   - Cumulative adoption chart for our product
#   - Competitor Product Phase-out Dates
#   - Insights / Action Items
# -------------------------------------------------------------------

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta

import dash
import dash_bootstrap_components as dbc
from dash import Dash, html, dcc, dash_table, Input, Output
from statsmodels.tsa.statespace.sarimax import SARIMAX

# -------------------------------------------------------------------
# 1) Synthetic Dataset Construction
# -------------------------------------------------------------------
PRODUCT_NAMES = ["OUR_BRAND"] + [f"COMP{i}_BRAND" for i in range(1, 10)]
np.random.seed(123)

# Monthly sales data
dates = pd.date_range("2022-01-31", periods=24, freq="ME")
product_data = {}
for product in PRODUCT_NAMES:
    sales = np.random.randint(100, 400, len(dates))
    product_data[product] = pd.Series(sales, index=dates)

# Phase-out dates for competitors (our brand = ongoing)
today = pd.to_datetime("2025-01-15")
phaseout_info = []
for prod in PRODUCT_NAMES[1:]:
    offset_months = np.random.randint(3, 24)
    phaseout_date = today + relativedelta(months=offset_months)
    phaseout_info.append({"Product": prod, "PhaseOutDate": phaseout_date.date()})

phaseout_df = pd.DataFrame(phaseout_info)

def likely_phaseout(date, reference):
    delta = relativedelta(date, reference)
    months = delta.years * 12 + delta.months
    return "YES" if months <= 6 else "NO"

phaseout_df["LikelyPhaseOut"] = phaseout_df["PhaseOutDate"].apply(
    lambda d: likely_phaseout(datetime.combine(d, datetime.min.time()), today))

# Adoption data for our brand
adopt_dates = pd.date_range("2023-06-30", periods=6, freq="QE")
adopt_values = [300, 620, 890, 1120, 1390, 1650]

# -------------------------------------------------------------------
# 2) Plotly + Dash
# -------------------------------------------------------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

def build_sales_forecast(product_name):
    series = product_data[product_name]
    model = SARIMAX(series, order=(1, 0, 1), seasonal_order=(1, 0, 0, 12),
                    enforce_stationarity=False, enforce_invertibility=False)
    fit = model.fit(disp=False)
    steps = 6
    last_date = series.index[-1]
    fc_index = pd.date_range(last_date + relativedelta(months=1), periods=steps, freq="ME")
    forecast_res = fit.get_forecast(steps=steps)
    fc_vals = forecast_res.predicted_mean
    ci = forecast_res.conf_int(alpha=0.2)
    ci.index = fc_index

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=series.index, y=series.values,
                             mode="lines+markers", name="Historical Sales"))
    fig.add_trace(go.Scatter(x=fc_index, y=fc_vals,
                             mode="lines+markers", name="Forecast", line=dict(dash="dash")))
    fig.add_trace(go.Scatter(x=fc_index, y=ci.iloc[:, 0], mode="lines", line=dict(width=0), showlegend=False))
    fig.add_trace(go.Scatter(x=fc_index, y=ci.iloc[:, 1], mode="lines", line=dict(width=0),
                             fill="tonexty", fillcolor="rgba(0,100,200,0.15)", name="80% CI"))
    fig.update_layout(title=f"6-Month Sales Forecast — {product_name}", height=320,
                      margin=dict(l=30, r=10, t=30, b=30))
    return fig

fig_adopt = go.Figure()
fig_adopt.add_trace(go.Scatter(x=adopt_dates, y=adopt_values, mode="lines+markers"))
fig_adopt.update_layout(title="Cumulative Customer Adoption — OUR_BRAND", height=320,
                        margin=dict(l=30, r=10, t=40, b=30))

# KPI Cards
kpi_dict = {
    "Total Sales (USD k)": 1980,
    "YTD Growth": "9.1%",
    "Units Sold YTD": 45000,
    "Retail Presence": "250 Stores"
}
kpi_cards = dbc.Row([
    dbc.Col(dbc.Card(dbc.CardBody([
        html.P(k, className="text-muted small"),
        html.H4(str(v), className="card-title")
    ]), className="text-center"), md=3, sm=6) for k, v in kpi_dict.items()
])

phaseout_table = dash_table.DataTable(
    data=phaseout_df.to_dict("records"),
    columns=[
        {"name": "Product", "id": "Product"},
        {"name": "Phase-Out Date", "id": "PhaseOutDate"},
        {"name": "Likely Phase-Out Soon?", "id": "LikelyPhaseOut"},
    ],
    style_cell={"fontSize": 12, "padding": "6px"},
    style_header={"backgroundColor": "#f7f7f7", "fontWeight": "bold"},
    page_size=10
)

action_items = [
    "Promote OUR_BRAND in regions where competitors are phasing out.",
    "Offer limited-time discounts ahead of competitor exit windows.",
    "Track inventory in high churn areas.",
    "Plan focused marketing where adoption is trending."
]

# -------------------------------------------------------------------
# 3) Layout
# -------------------------------------------------------------------
app.layout = dbc.Container(fluid=True, children=[
    html.H3("Supermarket Product Sales Dashboard — OUR_BRAND vs Competitors", className="mt-3 mb-4"),
    kpi_cards,
    html.Hr(),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Select Product for Sales Forecast"),
            dbc.CardBody([
                dcc.Dropdown(id="product-dropdown",
                             options=[{"label": n, "value": n} for n in PRODUCT_NAMES],
                             value="OUR_BRAND", clearable=False),
                dcc.Graph(id="sales-forecast-chart", style={"marginTop": 10})
            ])
        ]), md=6),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Customer Adoption Trend (Our Brand)"),
            dbc.CardBody([
                dcc.Graph(figure=fig_adopt, config={"displayModeBar": False})
            ])
        ]), md=6)
    ], className="mb-4"),

    dbc.Row([
        dbc.Col(dbc.Card([
            dbc.CardHeader("Competitor Product Phase-Outs"),
            dbc.CardBody([phaseout_table])
        ]), md=8),

        dbc.Col(dbc.Card([
            dbc.CardHeader("Insights"),
            dbc.CardBody([html.Ul([html.Li(x) for x in action_items], style={"fontSize": "0.9rem"})])
        ]), md=4)
    ])
], style={"maxWidth": "1400px"})

# -------------------------------------------------------------------
# 4) Callbacks
# -------------------------------------------------------------------
@app.callback(
    Output("sales-forecast-chart", "figure"),
    Input("product-dropdown", "value")
)
def update_forecast_chart(selected_product):
    return build_sales_forecast(selected_product)

# -------------------------------------------------------------------
# 5) Main
# -------------------------------------------------------------------
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=8054, debug=True)